In [12]:
import random
import json
from typing import Annotated, TypedDict, Literal
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage,BaseMessage, ToolMessage, AIMessage
from langgraph.graph import StateGraph, END, add_messages
from langgraph.prebuilt import ToolNode

In [13]:
model = ChatOpenAI(
    base_url="https://ws-02.wade0426.me/v1",
    api_key="day3hw",
    model="Qwen/Qwen3-VL-8B-Instruct",
    temperature=0.1
)

In [14]:
@tool
def get_weather(city: str):
    """查詢指定城市的天氣。輸入參數 city 必須是城市名稱。"""
    if random.random() < 0.5:
        return "系統錯誤"
    if "台北" in city:
        return "台北下大雨，氣溫 18 度"
    elif "台中" in city:
        return "台中天晴，氣溫 26 度"
    elif "高雄" in city:
        return "高雄多雲，氣溫 30 度"
    else:
        return "資料庫沒有這個程式的資料"


In [15]:
tools = [get_weather]
llm_with_tools = model.bind_tools(tools)

class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [16]:
def chatbot_node(state: AgentState):
    """思考節點"""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

tool_node_executor = ToolNode(tools)

def fallback_node(state: AgentState):
    """
    [新增] 被源節點：當嘗試次數過多時執行。
    """

    last_message =  state["messages"][-1]
    tool_call_id = last_message.tool_calls[0]["id"]

    error_message = ToolMessage(
        content="系統警示：以達到最大重試次數 (Max Retries Reached)。請停止嘗試，並告知使用者服務暫時無法使用。",
        tool_call_id=tool_call_id
    )
    return {"messages": [error_message]}

In [17]:
def router(state: AgentState) -> Literal["tools", "fallback", "end"]:
    messages = state["messages"]
    last_message = messages[-1]

    if not last_message.tool_calls:
        return "end"

    retry_count = 0

    for msg in reversed(messages[:-1]):
        if isinstance(msg, ToolMessage):
            if "系統錯誤" in msg.content:
                retry_count += 1
            else:
                break
        elif isinstance(msg, HumanMessage):
            break
    print(f"Debug: 目前連續重試次數: {retry_count}")

    if retry_count >= 3:
        return "fallback"
    return "tools"

In [18]:
workflow = StateGraph(AgentState)
workflow.add_node("agent", chatbot_node)
workflow.add_node("tools", tool_node_executor)
workflow.add_node("fallback", fallback_node)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    router,
    {
        "tools": "tools",
        "fallback": "fallback",
        "end": END
    }
)
workflow.add_edge("tools", "agent")
workflow.add_edge("fallback", "agent")

app = workflow.compile()
print(app.get_graph().draw_ascii())

                   +-----------+                    
                   | __start__ |                    
                   +-----------+                    
                          *                         
                          *                         
                          *                         
                     +-------+                      
                     | agent |*                     
                  ...+-------+ ***                  
              ....        *       ****              
           ...            *           ***           
         ..               *              **         
+---------+         +----------+         +-------+  
| __end__ |         | fallback |         | tools |  
+---------+         +----------+         +-------+  


In [20]:
if __name__ == "__main__":
    while True:
        uer_input = input("\nUser: ")
        if uer_input.lower() in ["exit", "quit"]: break
        for evert in app.stream({"messages": [HumanMessage(content=uer_input)]}):
            for key, value in evert.items():
                if key == "agent":
                    msg = value["messages"][-1]
                    if msg.tool_calls:
                        print(f" -> [Agent]: 決定呼叫工具 （重試中..)")
                    else:
                        print(f" -> [Agent]: {msg.content}")
                elif key == "tools":
                    print(f" -> [Tools]: 系統錯誤... (失敗）")
                elif key == "fallback":
                    print(f" -> [Fallback] 觸發中斷機制：停止重試")


 -> [Agent]: 你好！請問有什麼我可以幫忙的嗎？
Debug: 目前連續重試次數: 0
 -> [Agent]: 決定呼叫工具 （重試中..)
 -> [Tools]: 系統錯誤... (失敗）
 -> [Agent]: 台北下大雨，氣溫 18 度。


KeyboardInterrupt: Interrupted by user